In [30]:
import tensorflow as tf
import sys

print("TensorFlow Version:", tf.__version__)
print("Python Version:", sys.version)


TensorFlow Version: 2.17.0
Python Version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


In [34]:
!pip install tensorflow-addons


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons


In [62]:
import os
import numpy as np
import collections
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt 



In [96]:
import os
import shutil
import random

# Paths to your original and new balanced datasets
original_dataset_dir = r'C:\Users\Raj Kumar\Downloads\archive (6)\Datasets'
balanced_dataset_dir = r'C:\Users\Raj Kumar\Downloads\Balanced_Datasets'

# Create the balanced dataset directory
if not os.path.exists(balanced_dataset_dir):
    os.makedirs(balanced_dataset_dir)

# Loop over all classes (subdirectories in original dataset)
class_names = os.listdir(original_dataset_dir)
max_class_size = 0

# Find the class with the maximum number of images
for class_name in class_names:
    class_dir = os.path.join(original_dataset_dir, class_name)
    if os.path.isdir(class_dir):
        num_images = len(os.listdir(class_dir))
        if num_images > max_class_size:
            max_class_size = num_images

# Now, create the balanced dataset by copying images
for class_name in class_names:
    class_dir = os.path.join(original_dataset_dir, class_name)
    if os.path.isdir(class_dir):
        # Create a directory for the current class in the balanced dataset
        balanced_class_dir = os.path.join(balanced_dataset_dir, class_name)
        if not os.path.exists(balanced_class_dir):
            os.makedirs(balanced_class_dir)
        
        # Get all image paths in the current class
        image_paths = [os.path.join(class_dir, fname) for fname in os.listdir(class_dir) if fname.endswith(('.jpg', '.jpeg', '.png'))]
        
        # Calculate how many images to copy
        num_images_to_copy = max_class_size - len(image_paths)
        
        # If the class has fewer images, duplicate random images to balance
        if num_images_to_copy > 0:
            # Randomly sample images to duplicate
            additional_images = random.choices(image_paths, k=num_images_to_copy)
            image_paths += additional_images
        
        # Copy the images to the balanced dataset directory
        for image_path in image_paths:
            shutil.copy(image_path, balanced_class_dir)

print("Dataset has been balanced and copied to:", balanced_dataset_dir)


Dataset has been balanced and copied to: C:\Users\Raj Kumar\Downloads\Balanced_Datasets


In [97]:
original_dataset_dir = Path(r"C:\Users\Raj Kumar\Downloads\archive (6)\Datasets")  # Original dataset path
balanced_dataset_dir = Path(r"C:\Users\Raj Kumar\Downloads\Balanced_Dataset")  # New balanced dataset path

# Create new directory if it doesn't exist
balanced_dataset_dir.mkdir(parents=True, exist_ok=True)


In [98]:
class_counts = {}

# Loop through each class folder
for class_name in os.listdir(original_dataset_dir):
    class_path = original_dataset_dir / class_name
    if os.path.isdir(class_path):
        class_counts[class_name] = len(os.listdir(class_path))

# Find the maximum number of images in any class
max_images = max(class_counts.values())

print("Original class distribution:", class_counts)
print(f"Max images in any class: {max_images}")


Original class distribution: {'fifty_new': 265, 'fifty_old': 234, 'five_hundred': 279, 'hundred_new': 224, 'hundred_old': 107, 'ten_new': 345, 'ten_old': 306, 'twenty_new': 288, 'twenty_old': 257, 'two_hundred': 272, 'two_thousand': 102}
Max images in any class: 345


In [99]:
for class_name, count in class_counts.items():
    src_class_path = original_dataset_dir / class_name
    dest_class_path = balanced_dataset_dir / class_name
    dest_class_path.mkdir(parents=True, exist_ok=True)

    # Copy existing images
    for img_name in os.listdir(src_class_path):
        shutil.copy(src_class_path / img_name, dest_class_path / img_name)

    # If class is underrepresented, duplicate random images
    if count < max_images:
        extra_images_needed = max_images - count
        images = os.listdir(dest_class_path)
        
        for i in range(extra_images_needed):
            img_to_copy = random.choice(images)  # Pick a random image
            new_img_name = f"copy_{i}_{img_to_copy}"  # Rename copied image
            shutil.copy(dest_class_path / img_to_copy, dest_class_path / new_img_name)

print("Balanced dataset created successfully!")


Balanced dataset created successfully!


In [100]:
balanced_class_counts = {class_name: len(os.listdir(balanced_dataset_dir / class_name)) 
                         for class_name in os.listdir(balanced_dataset_dir)}

print("Balanced class distribution:", balanced_class_counts)


Balanced class distribution: {'fifty_new': 345, 'fifty_old': 345, 'five_hundred': 345, 'hundred_new': 345, 'hundred_old': 345, 'ten_new': 345, 'ten_old': 345, 'twenty_new': 345, 'twenty_old': 345, 'two_hundred': 345, 'two_thousand': 345}


In [101]:
# Path to the new balanced dataset
DATASET_PATH = Path(r"C:\Users\Raj Kumar\Downloads\Balanced_Dataset")

# Image parameters
IMAGE_SIZE = (128, 128)  # Adjust size based on model needs
BATCH_SIZE = 32
EPOCHS = 20


In [102]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)

# Load training data
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data
val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Get number of classes
num_classes = len(train_data.class_indices)
print(f"Detected {num_classes} classes: {train_data.class_indices}")


Found 3036 images belonging to 11 classes.
Found 759 images belonging to 11 classes.
Detected 11 classes: {'fifty_new': 0, 'fifty_old': 1, 'five_hundred': 2, 'hundred_new': 3, 'hundred_old': 4, 'ten_new': 5, 'ten_old': 6, 'twenty_new': 7, 'twenty_old': 8, 'two_hundred': 9, 'two_thousand': 10}


In [103]:
model = Sequential([
    # Convolutional layers
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten and Dense layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevents overfitting
    Dense(num_classes, activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model summary
model.summary()


C:\Users\Raj Kumar\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,059 (12.61 MB)

 Trainable params: 3,306,059 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [104]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    verbose=1
)


Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.0966 - loss: 2.3987 - val_accuracy: 0.0870 - val_loss: 2.4212
Epoch 2/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.1589 - loss: 2.3197 - val_accuracy: 0.1528 - val_loss: 2.3073
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.2367 - loss: 2.1438 - val_accuracy: 0.1897 - val_loss: 2.1792
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.2837 - loss: 2.0319 - val_accuracy: 0.1910 - val_loss: 2.1379
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.3371 - loss: 1.9183 - val_accuracy: 0.2596 - val_loss: 2.0358
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.3433 - loss: 1.8502 - val_accuracy: 0.2754 - val_loss: 2.0147
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.3772 - loss: 1.7923 - val_accuracy: 0.3057 - val_loss: 1.9498
Epoch 8/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.3792 - loss: 1.7026 - val_accuracy: 0.3373 - v

In [105]:
model.save("currency_detector4.h5")
print("Model saved successfully!")


Model saved successfully!


In [106]:
loss, accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 454ms/step - accuracy: 0.4403 - loss: 1.5913
Validation Accuracy: 41.37%


In [116]:
print(train_data.class_indices)


{'fifty_new': 0, 'fifty_old': 1, 'five_hundred': 2, 'hundred_new': 3, 'hundred_old': 4, 'ten_new': 5, 'ten_old': 6, 'twenty_new': 7, 'twenty_old': 8, 'two_hundred': 9, 'two_thousand': 10}


In [117]:
currency_labels = [
    "₹50", "₹50", "₹500", "₹100", "₹100", 
    "₹10", "₹10", "20", "20", "200", "200"
]


In [ ]:
from tensorflow.keras.preprocessing import image

In [121]:
# Function to preprocess the image
def preprocess_image(img_path, target_size=(128, 128)):
    img = image.load_img(img_path, target_size=target_size)  # Load image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims for batch
    img_array = img_array / 255.0  # Normalize
    return img_array

In [122]:
def predict_currency(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Get class with highest probability
    confidence = np.max(prediction)  # Get confidence score
    
    currency_name = currency_labels[predicted_class]  # Map index to currency name
    print(f"Predicted Currency: {currency_name} (Confidence: {confidence * 100:.2f}%)")
    return currency_name, confidence


In [135]:
img_path = r"C:\Users\Raj Kumar\Downloads\notes\2001.jpg"  # Change as needed
predict_currency(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Predicted Currency: ₹100 (Confidence: 37.26%)


('₹100', 0.3726126)

In [107]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from pathlib import Path

In [108]:
model = load_model("currency_detector4.h5")
print("Model loaded successfully!")


Model loaded successfully!


In [109]:
# Define the class labels (same order as in train_data.class_indices)
class_labels = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 
                'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_10']

# You should replace the above list with your actual class names

In [110]:
def preprocess_image(img_path, target_size=(128, 128)):
    img = image.load_img(img_path, target_size=target_size)  # Load image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims for batch
    img_array = img_array / 255.0  # Normalize
    return img_array


In [111]:
def predict_currency(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Get class with highest probability
    confidence = np.max(prediction)  # Get confidence score
    
    print(f"Predicted Class: {class_labels[predicted_class]} (Confidence: {confidence * 100:.2f}%)")
    return class_labels[predicted_class], confidence


In [115]:
img_path = r"C:\Users\Raj Kumar\Downloads\5001.jpg"  # Replace with actual path
predicted_class, confidence = predict_currency(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Predicted Class: class_1 (Confidence: 53.28%)
